In [1]:
# Hack to make the module importable
import sys
sys.path.append(r'./../')

In [1]:
%load_ext autoreload
%autoreload 2
from py2neo import Graph, NodeMatcher
import pandas as pd

from rel2graph.relational_modules.pandas import PandasDataframeIterator
from rel2graph import IteratorIterator
from rel2graph import Converter

filename = "./pd_dataframe_demo_config.yaml"

In [2]:
# Configure Logging
import logging

logging.basicConfig(level=logging.WARNING)
logging.getLogger("rel2graph").setLevel(logging.DEBUG)

In [3]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [4]:
data = {
  "ID": [1,2,2,3,4,4],
  "FirstName": ["Julian", "Fritz",  "Fritz", "Hans", "Rudolfo", "Rudolfo"],
  "LastName": ["Minder", "Generic", "SomeGuy", "Müller", "Muster", "Muster"],
  "FavoriteFlower": ["virginica", "setosa", "setosa", "versicolor", "setosa", "setosa"]
}
people = pd.DataFrame(data)
people

,ID,FirstName,LastName,FavoriteFlower
0,1,Julian,Minder,virginica
1,2,Fritz,Generic,setosa
2,2,Fritz,SomeGuy,setosa
3,3,Hans,Müller,versicolor
4,4,Rudolfo,Muster,setosa
5,4,Rudolfo,Muster,setosa


In [5]:
graph = Graph(scheme="http", host="localhost", port=7474,  auth=('neo4j', 'password')) ## UPDATE password

graph.delete_all()  # reset graph (only wehn first creating the databse, here for debugging purposes)

In [6]:
iterator = IteratorIterator([PandasDataframeIterator(people, "Person"), PandasDataframeIterator(iris, "Flower")])

In [7]:
converter = Converter(filename, iterator, graph, num_workers=5)

DEBUG:rel2graph.core.config_parser:Parsing config file './pd_dataframe_demo_config.yaml'
DEBUG:rel2graph.core.config_parser:Compiling ENTITY("Flower") : {'NODE("Flower") flower': ['sepal_length = Flower.sepal_length', 'sepal_width = Flower.sepal_width', 'petal_length = Flower.petal_width', 'petal_width = Flower.petal_width'], 'NODE("Species", "BioEntity") species': ['+ Name = Flower.species'], 'RELATION(flower, "is", species)': None}
DEBUG:rel2graph.core.config_parser:Compiling module 'AttributeFactory' with static args ['sepal_length', 'sepal_length']
DEBUG:rel2graph.core.factories.registrar:Requested factory 'AttributeFactory' from registry.
DEBUG:rel2graph.core.config_parser:Compiling module 'AttributeFactory' with static args ['sepal_width', 'sepal_width']
DEBUG:rel2graph.core.factories.registrar:Requested factory 'AttributeFactory' from registry.
DEBUG:rel2graph.core.config_parser:Compiling module 'AttributeFactory' with static args ['petal_length', 'petal_width']
DEBUG:rel2graph.

In [8]:
from tqdm.notebook import tqdm
converter(progress_bar=tqdm)

  0%|          | 0/312 [00:00<?, ?it/s]

INFO:rel2graph.core.converter:Running convertion with 5 parallel workers.
INFO:rel2graph.core.converter:Starting creation of nodes.
DEBUG:rel2graph.core.converter:Starting Worker 0
DEBUG:rel2graph.core.converter:Starting Worker 1
DEBUG:rel2graph.core.converter:Worker 0: Processing resource type: 'Person'
DEBUG:rel2graph.core.converter:Starting Worker 2
DEBUG:rel2graph.core.converter:Starting Worker 3
DEBUG:rel2graph.core.converter:Starting Worker 4
DEBUG:rel2graph.core.converter:Worker 1: Processing resource type: 'Person'
DEBUG:rel2graph.core.converter:Worker 2: Processing resource type: 'Person'
DEBUG:rel2graph.core.converter:Worker 3: Processing resource type: 'Person'
DEBUG:rel2graph.core.converter:Worker 4: Processing resource type: 'Person'
DEBUG:rel2graph.core.converter:Worker 0: Processing resource type: 'Person'
DEBUG:rel2graph.core.converter:Worker 1: Processing resource type: 'Flower'
DEBUG:rel2graph.core.converter:Worker 2: Processing resource type: 'Flower'
DEBUG:rel2graph